In [1]:
# --- 2.1 Data Cleaning ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load cleaned data from Phase 1
df = pd.read_csv('../data/processed/cleaned_data.csv')
df['Date'] = pd.to_datetime(df['Date'])  # Ensure datetime

# Re-check missing values
print("Missing values:\n", df.isnull().sum())

# Drop rows with missing data (or you can fill based on logic)
df = df.dropna()

# Remove outliers using IQR method (optional)
def remove_outliers_iqr(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return dataframe[(dataframe[column] >= lower) & (dataframe[column] <= upper)]

for col in ['Age', 'Price per Unit', 'Quantity', 'Total Amount']:
    df = remove_outliers_iqr(df, col)

# Standardize date format
df['Date'] = pd.to_datetime(df['Date'])

df.head()


Missing values:
 Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
Month               0
dtype: int64


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Month
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,2023-11
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,2023-02
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,2023-01
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,2023-05
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,2023-05


In [2]:
# Time-based features
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter
df['Year'] = df['Date'].dt.year
df['Is_Weekend'] = df['Weekday'].isin(['Saturday', 'Sunday'])

# Season (simple categorization)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(get_season)


In [3]:
# Daily Sales grouped
daily_sales = df.groupby('Date')['Total Amount'].sum().reset_index()
daily_sales['Rolling_7D'] = daily_sales['Total Amount'].rolling(window=7).mean()
daily_sales['Lag_1'] = daily_sales['Total Amount'].shift(1)
daily_sales['Lag_7'] = daily_sales['Total Amount'].shift(7)

# Merge back (optional)
df = df.merge(daily_sales, how='left', on='Date', suffixes=('', '_Daily'))

# Drop NA rows from lags if present
df = df.dropna()


In [4]:
# Purchase frequency per customer
cust_freq = df.groupby('Transaction ID').size().reset_index(name='Purchase_Frequency')
df = df.merge(cust_freq, on='Transaction ID', how='left')

# Average spend per transaction
df['Average_Spend'] = df['Total Amount'] / df['Quantity']


In [ ]:
# Product category popularity
category_popularity = df['Product Category'].value_counts().to_dict()
df['Category_Popularity'] =  df['Product Category'].map(category_popularity)


In [6]:
continuous_vars = ['Customer Age', 'Price per Unit', 'Quantity', 'Total Amount', 'Purchase_Frequency']
print("Continuous variables to be binned:\n", continuous_vars)


Continuous variables to be binned:
 ['Customer Age', 'Price per Unit', 'Quantity', 'Total Amount', 'Purchase_Frequency']


In [8]:
df.to_csv('../data/processed/original_features.csv', index=False)
